In [249]:
from os import listdir
from os.path import join
import pandas as pd
import numpy as np
import pickle
import re
from bs4 import BeautifulSoup
import tldextract

In [386]:
print('IMPORTANT - IF THE SHAPE OF THE LOADED DATAFRAME CHANGES (5404 rows), THE MANUAL EDITS WILL CREATE MISTAKES')

IMPORTANT - IF THE SHAPE OF THE LOADED DATAFRAME CHANGES (5404 rows), THE MANUAL EDITS WILL CREATE MISTAKES


# 1.Get variables and load dataframe

In [387]:
project_folder = join('/Users','Toavina','githubdata')
hn_df_load_folder = join('5.hn_postings_dl','2.pickles','6.aggregate_dataframe')
hn_df_filename = 'hn_posts_agg_df.pkl'

cleanup_load_folder = '1.manual_clean_files'
ghlink_filename = 'hn_df_emails_manual_changes.xlsx'

save_folder = join('2.cleaned_df')
save_filename = 'cleaned_df.pkl'

In [297]:
print('Loading HN CV posts dataframe')
hn_df = pickle.load(open(join(project_folder,hn_df_load_folder,hn_df_filename),'rb'))

Loading HN CV posts dataframe


# 2.Amend Github account links that were changed manually in Excel

In [299]:
print('Loading Excel file to cleanup Github links from Hacker News dataframe')
email_clean = pd.read_excel(join(cleanup_load_folder,ghlink_filename), header=0, index_col=0)
email_clean.index.name=None

Loading Excel file to cleanup Github links from Hacker News dataframe


In [300]:
# Removing columns to prevent conflicts, will then be merged by index
email_clean = email_clean[['corrected_gh','manually_changed_gh']]

In [301]:
print('Merging dataframes')
hn_df = hn_df.join(email_clean)

Merging dataframes


In [302]:
print('Replacing manually changed Github account links')
def apply_manual_gh_change(row):
    if row['manually_changed_gh'] == 1:
        row['github_account'] = row['corrected_gh']
    return row

Replacing manually changed Github account links


In [303]:
hn_df = hn_df.apply(apply_manual_gh_change, axis=1)

In [304]:
print('Removing manual email cleanup columns')

hn_df = hn_df.drop(['corrected_gh','manually_changed_gh'], axis = 1)

Removing manual email cleanup columns


# 3.Programmatically cleaning Github accounts 

In [306]:
def change_str(string):
    
    if string == None:
        pass
    else:
        try:
            string = str(string)

            # Replace username with https://github.com/username
            string = re.sub('^(?!.*(github|http))','https://github.com/', string)
            
            # Replace github.com with https://github.com
            string = re.sub(r'^(github.com)\/', 'https://github.com/' , string)

            # Replace http with https
            string = re.sub(r'tp:','tps:', string)

            # Replace http://www. with https://
            string = re.sub(r'www.','', string)
        

            # Replace https:github.com with https://github.com
            string = re.sub(r'https\:github.com', 'https://github.com', string)

            # Replace @username with https://github.com/username
            string = re.sub('@','https://github.com/', string)

            # Remove [
            string = re.sub(r'\[\s+', '' , string)

            # Remove ]
            string = re.sub(r'\s+\]','', string)
            
            # Remove extranuous non-user links
            if 'https://github.com' in string and string.count('/') >= 4:
                split_str = string.split('/')
                string = 'https://github.com/' + split_str[3]
            

            # Manual changes
            string = re.sub(r'https://github.com/dt1/omark','https://github.com/dt1',string)
            string = re.sub('fractalide/fractalide','https://github.com/fractalide',string) 
        
        except:
            pass

    return string


In [307]:
print('Programatically changing github_account column to rectify some mistakes')
hn_df['github_account'] = hn_df['github_account'].apply(change_str)

Programatically changing github_account column to rectify some mistakes


# 4. Trying to find Github links to add other users

In [309]:
# Create a dataframe filtering for rows with no discernible Github account 
hn_nogh_accnt = hn_df[pd.isnull(hn_df['github_account'])]

In [310]:
def find_github_in_links(df):
    """Returns a list of indices where the word github appears in the list"""
    gh_in_links = []
    for index, value in hn_nogh_accnt['links'].items():
        if 'github' in str(value):
            gh_in_links.append(index)
    gh_inlinks = sorted(list(set(gh_in_links)))
    return gh_in_links

In [311]:
gh_in_links = find_github_in_links(hn_nogh_accnt)

In [312]:
hn_nogh_accnt['soup'] = hn_nogh_accnt['links']

/Users/Toavina/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [313]:
def change_to_soup(item):
    try:
        item = str(item)
        item = BeautifulSoup(item, 'lxml')
    except:
        pass
    return item

In [314]:
hn_nogh_accnt['soup'] = hn_nogh_accnt['soup'].apply(change_to_soup)

/Users/Toavina/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [315]:
hn_gh_in_links = hn_nogh_accnt.loc[gh_in_links]

In [316]:
def get_gh_links(soup):
    
    def github_href(href):
        """Returns only links in the Soup that have github in them"""
        return href and re.compile('github').search(href)
    
    new_val = soup.find_all('a',href=github_href)[0]['href']
    
    return new_val

In [317]:
hn_gh_in_links['gh_links'] = hn_gh_in_links['soup'].apply(get_gh_links) 

In [318]:
print('Deleting unused column')
if 'soup' in hn_gh_in_links.columns:
    del hn_gh_in_links['soup']
else:
    pass

Deleting unused column


In [319]:
def change_github_io_to_com(string):
    if 'github.io' in string:
        string = 'https://github.com/'+tldextract.extract(string).subdomain
    
    # Remove multiple / after username
    if string.count('/') >= 4:
        split_str = string.split('/')
        string = 'https://github.com/' + split_str[3]
        
    # Change gist.github.com to github.com
    string = re.sub(r'https://gist.github.com','https://github.com',string)
    string = re.sub(r'http://gist.github.com','https://github.com',string)
        
    # Remove ?tab=repositories
    string = re.sub(r'\?tab=repositories','',string)
    
    return string

In [320]:
print('Amending some names for easier extraction')
hn_gh_in_links['gh_links'] = hn_gh_in_links['gh_links'].apply(change_github_io_to_com)
hn_gh_in_links['gh_links'] = hn_gh_in_links['gh_links'].apply(change_str)

Amending some names for easier extraction


In [321]:
print('Joining new GH links with main dataframe')

Joining new GH links with main dataframe


In [322]:
hn_gh_in_links = hn_gh_in_links[['gh_links']]

In [323]:
hn_df = hn_df.join(hn_gh_in_links)

In [324]:
print('Adding new GH users to github_account column')

Adding new GH users to github_account column


In [325]:
def add_new_GH_accounts(row):
    
    if pd.isnull(row['github_account']) and row['gh_links'] != np.nan:
        row['github_account'] = row['gh_links']
    
    return row

In [326]:
hn_df = hn_df.apply(add_new_GH_accounts, axis=1)

In [327]:
print('removing gh_links column as unused from now on')
if 'gh_links' in hn_df.columns:
    del hn_df['gh_links']

removing gh_links column as unused from now on


# 5 Picking up Github in resume

In [329]:
print('Creating slice with resumes that mention linkedin')
hn_df_in_resume = hn_df[hn_df['resume'].str.contains('github')==True]

Creating slice with resumes that mention linkedin


In [330]:
print('There are ' + str(len(hn_df_in_resume[pd.isnull(hn_df_in_resume['github_account'])])) + \
      ' rows which mention Github in the resume for which there the github_account column is empty')

There are 71 rows which mention Github in the resume for which there the github_account column is empty


In [331]:
resumes_to_process_excel_filename = 'resumes_to_process.xlsx'

In [332]:
print("Dumping those rows' resume column to be manually processed as " + \
      join(cleanup_load_folder,resumes_to_process_excel_filename))

Dumping those rows' resume column to be manually processed as 1.manual_clean_files/resumes_to_process.xlsx


In [333]:
resumes_to_process = pd.DataFrame(hn_df_in_resume[pd.isnull(hn_df_in_resume['github_account'])]['resume'])

In [334]:
writer = pd.ExcelWriter(join(cleanup_load_folder,resumes_to_process_excel_filename), engine='xlsxwriter')

In [335]:
resumes_to_process.to_excel(writer, sheet_name='Sheet1')

In [336]:
print('Loading cleaned up excel as a dataframe')
cleaned_up_filename = 'resumes_to_process_v1.xlsx'
gh_clean = pd.read_excel(join(cleanup_load_folder,cleaned_up_filename), header=0, index_col=0)
gh_clean.index.name=None

Loading cleaned up excel as a dataframe


In [337]:
print('joining dataframes on index')

joining dataframes on index


In [338]:
hn_df = hn_df.join(gh_clean)

In [340]:
print('Updating github account column with additional users')

def update_github_column(row):
    """Updates the github account column with additional user details"""
    if pd.isnull(row['github_account']):
        if row['changed_resume'] != np.nan:
            row['github_account'] = row['changed_resume']
    else:
        pass
    return row

hn_df = hn_df.apply(update_github_column, axis=1)

Updating github account column with additional users


In [369]:
print('Deleting unused columns')

if 'original_resume' in hn_df.columns and 'changed_resume' in hn_df.columns:
    hn_df = hn_df.drop(['original_resume','changed_resume'], axis=1)

Deleting unused columns


In [113]:
#TODO - Change join to join on original_resume in case original dataframe changes

# 6. Seeing if can find Github usernames from text

In [359]:
other_potential_gh_account_num = len(hn_df[(hn_df['github_mention'] == True) & (pd.isnull(hn_df['github_account']))])
print('There are ' +str(other_potential_gh_account_num) +' Github mentions without a link that could also be '\
      +'searched with enough time')

# TODO - If have time, look at those users to extract additional - marginal gain given only 95 users

There are 95 Github mentions without a link that could also be searched with enough time


# 7. Inferring Github usernames from email address by matching with downloaded GH users

In [360]:
# TODO - Match email address from GH user database with email to see how many additional Github accounts I can get

# 8. Inferring Github usernames from username (if relevant, mark those)

In [361]:
# TODO for more data - Find matching usernames from HN and Github and see if can find a match

# 9. Bonus - If can identify from LN profile and their name

In [362]:
# Lots of work, but for those that don't have GH username but post a LinkedIn profile - get their LinkedIn profile,
# get their name, and then see if that matches a name on the Github database to identify additional accounts

# 10. Inferring Github usernames from link

In [363]:
print('Creating column to extract inferred github username')

Creating column to extract inferred github username


In [364]:
def infer_gh_username(gh_link):
    
    if gh_link == None:
        pass
    else:
        try:
            gh_link = str(gh_link)
            
            gh_link = re.sub('.+github.com/','',gh_link)
            gh_link = re.sub('github.com/','', gh_link)
            gh_link = re.sub(r'/','', gh_link)
            
        except:
            pass
    
    return gh_link

In [365]:
hn_df['inferred_ghuser'] = hn_df['github_account'].apply(infer_gh_username)

In [384]:
print('There are ' +str(len(hn_df[hn_df['inferred_ghuser'] != 'nan']['inferred_ghuser'].unique())) +\
      ' unique GH users identified that also have an HN account')

There are 918 unique GH users identified that also have an HN account


# 11. Pickling dataframe for next step

In [388]:
print('Pickling dataframe to ' + join(save_folder, save_filename))
pickle.dump(hn_df, open(join(save_folder, save_filename),'wb'))

Pickling dataframe to 2.cleaned_df/cleaned_df.pkl
